In [25]:
from typing import *
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo import UpdateOne
from pandas import DataFrame
from typing import List
import logging

logger = logging.getLogger(__name__)


class MongoClientConnector:
    def __init__(self, uri_mongo_connect: str):
        self.uri_mongo_connect = uri_mongo_connect
        self.mongo_client = MongoClient(self.uri_mongo_connect)

    def insert_data(self, database: str, collection_str: str, data: List[dict]):
        collection = self.mongo_client[database][collection_str]
        collection.insert_many(data)
        logger.info(f"Insert to {database}/{collection_str} with {len(data)} records")

    def filter(self, database: str, collection: str, dict_filter:dict):
        collection = self.mongo_client[database][collection]
        query_result = collection.find(dict_filter)
        return list(query_result)

    def get_all_documents(self, database: str, collection: str):
        return list(self.mongo_client[database][collection].find({}))

    def collection_update_one(self, database, collection, filter: dict, doc: dict):
        self.mongo_client[database][collection].replace_one(
            filter=filter,
            replacement=doc,
            upsert=True
        )

    def bulk_update_one(self, database: str, collection: str, list_filters: list, list_docs, **kwargs):
        try:
            requests = [UpdateOne(filter=filter_doc, update=doc, upsert=True) for filter_doc, doc in zip(list_filters, list_docs)]
            self.mongo_client[database][collection].bulk_write(requests)
        except Exception as e:
            logger.error(str(e))
            logger.info(list_filters)
            logger.info(list_docs)
            logger.info(f"{kwargs.get('batch_token_ids')}")



In [26]:
mongo_client = MongoClientConnector('mongodb://localhost:27017')

In [42]:
temps = mongo_client.get_all_documents(database='information', collection='tokenomic')

In [48]:
collection = mongo_client.mongo_client['price']['price_metadata_1D']
collection.drop()

In [32]:
list_tokens = mongo_client.get_all_documents(database='information', collection='token')
list_token_ids = [token['id'] for token in list_tokens]

In [33]:
type(list_token_ids[0])

int

In [39]:
import requests
from bs4 import BeautifulSoup
import csv

URL = "https://etherscan.io/token/0x8ab7404063ec4dbcfd4598215992dc3f8ec853d7#balances"
resp = requests.get(URL)
sess = requests.Session()
soup = BeautifulSoup(sess.get(URL).text, 'html.parser')

In [49]:
from tqdm import tqdm

In [38]:
for collection in tqdm(mongo_client.mongo_client['price'].list_collection_names()):
    if 'price' not in collection and int(collection.split('_')[0]) not in list_token_ids:
        print(collection)

In [51]:
for collection_name in tqdm(mongo_client.mongo_client['price'].list_collection_names()):
    collection = mongo_client.mongo_client['price'][collection_name]
    doc = collection.find_one(sort=[('date_timestamp', -1)])
    data = {
        'token_id': doc['token_id'],
        'token_name': doc['token_name'],
        'symbol': doc['symbol'],
        'max_date': doc['date_timestamp'],
        'max_date_str': doc['date_str'],
        'current_market_cap': doc['market_cap'],
        "current_volume": doc['volume'],
        'current_circulating_supply': doc['circulating_supply']
    }
    mongo_client.insert_data(database='price', collection_str='price_metadata_1D', data=[data])

  0%|          | 0/9352 [00:00<?, ?it/s]

100%|██████████| 9352/9352 [00:44<00:00, 209.59it/s]


In [14]:
from datetime import datetime 
from dateutil import parser

In [18]:
datetime.strptime('20230404', '%Y%m%d')

datetime.datetime(2023, 4, 4, 0, 0)

In [30]:
format_string = '%Y-%m-%dT%H:%M:%S.%fZ'

# Convert the string to a datetime object
dt_object = datetime.strptime('2024-03-31T06:04:10.774Z', format_string) - datetime.strptime('20240331', '%Y%m%d')

In [35]:
datetime.strptime('20240331', '%Y%m%d').timestamp()

1711818000.0

In [9]:
count number of collections in python mongod

In [38]:
'20230303' < '20230302'

False

In [13]:
list(collection.find({'token_name': 'SOL'}))

[{'_id': ObjectId('660823b3c70f5be1464437d3'),
  'token_id': 5426,
  'token_name': 'SOL',
  'max_date': '1711809300',
  'max_date_str': '20240330',
  'current_market_cap': 87405122768.67383,
  'current_volume': 3573349716.42},
 {'_id': ObjectId('660828c7c70f5be146502309'),
  'token_id': 16116,
  'token_name': 'SOL',
  'max_date': '1711810200',
  'max_date_str': '20240330',
  'current_market_cap': 0.0,
  'current_volume': 2240597284.98},
 {'_id': ObjectId('6608321cc70f5be146623c72'),
  'token_id': 28825,
  'token_name': 'SOL',
  'max_date': '1711812900',
  'max_date_str': '20240330',
  'current_market_cap': 0.0,
  'current_volume': 935.04},
 {'_id': ObjectId('6608321dc70f5be1466240f5'),
  'token_id': 28828,
  'token_name': 'SOL',
  'max_date': '1711812900',
  'max_date_str': '20240330',
  'current_market_cap': 0.0,
  'current_volume': 37.65},
 {'_id': ObjectId('660833d6c70f5be146664dbd'),
  'token_id': 29905,
  'token_name': 'SOL',
  'max_date': '1711813200',
  'max_date_str': '20240330

In [10]:
import requests

In [12]:
requests.get('https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories', params={
                'limit': 5000
            }, headers={'X-CMC_PRO_API_KEY': '6f44899b-429c-43cf-88c6-01aab25f050b', 'Accept': 'application/json', 'Accept-Encoding': 'deflate, gzip'}).json()['data']

[{'id': '6604f2405726cb6104472fdb',
  'name': 'Presale Memes',
  'title': 'Presale Memes',
  'description': 'Presale Memes',
  'num_tokens': 3,
  'avg_price_change': -7.307536085000001,
  'market_cap': 315992213.89,
  'market_cap_change': -4.046,
  'volume': 389965522.2642178,
  'volume_change': 1.7009000000000007,
  'last_updated': '2024-03-28T04:30:07.129Z'},
 {'id': '66039e3c6253043a6a580e65',
  'name': 'Move VM (MVM)',
  'title': 'Move VM (MVM)',
  'description': 'Move VM (MVM)',
  'num_tokens': 2,
  'avg_price_change': -0.377878885,
  'market_cap': 9129230248.3,
  'market_cap_change': -0.35750000000000004,
  'volume': 502976915.88380015,
  'volume_change': -30.97555,
  'last_updated': '2024-03-27T04:19:33.033Z'},
 {'id': '65fafbaccd69e615d550867e',
  'name': 'Atomicals Ecosystem',
  'title': 'Atomicals Ecosystem',
  'description': 'Atomicals Ecosystem',
  'num_tokens': 8,
  'avg_price_change': 3.4513321849999996,
  'market_cap': 0,
  'market_cap_change': 0,
  'volume': 1336013.325